# This project provides a Python solution for extracting relevant data from PDF invoices. It addresses challenges like multi-page documents and variable formats, ensuring a reliable extraction process.

## Understanding Requirements and Data Structure

 1. Header Information 
• Invoice Number: "123100401" 
o Format: Numeric, typically a string of numbers. 
• Invoice Date: "1. März 2024" 
o Format: Date in DD/MM/YYYY or MM/DD/YYYY. 
• Vendor Name: "CPB Software (Germany) GmbH" 
o Format: String. 
• Customer Name: "Musterkunde AG" 
o Format: String. 
• Customer Number: "12345" 
o Format: Numeric. 
2. Payment Information 
• Total Amount (without VAT): "381,12 €" 
o Format: Numeric with two decimal points, currency in Euro (€). 
• VAT (19%): "72,41 €" 
o Format: Numeric with two decimal points, currency in Euro (€). 
• Gross Amount (with VAT): "453,53 €" 
o Format: Numeric with two decimal points, currency in Euro (€). 
• IBAN: "DE29 1234 5678 9012 3456 78" 
o Format: Alphanumeric (German IBAN format). 
• BIC: "GENODE51MIC" 
o Format: Alphanumeric (SWIFT code). 
3. Invoice Period 
• Start Date: "01.02.2024" 
o Format: Date in DD/MM/YYYY or MM/DD/YYYY. 
• End Date: "29.02.2024" 
o Format: Date in DD/MM/YYYY or MM/DD/YYYY. 
4. Line Items / Transaction Fees 
Each line item includes several components: 
• Description: E.g., "Basic Fee wmView," "Transaction Fee T1." 
o Format: String. 
• Quantity: E.g., "1" for "Basic Fee wmView." 
o Format: Numeric. 
• Price: E.g., "130,00 €" for the basic fee. 
o Format: Numeric with two decimal points, currency in Euro (€). 
• Total for each fee/transaction
o Format: Numeric with two decimal points, currency in Euro (€). 
5. Variations and Notes 
• Language: Some fields  may appear in different languages
• Different Invoice Formats: The layout and structure may differ between vendors, 
particularly in the placement of totals, VAT, and line items. 
• Optional Fields: Certain invoices might contain additional details, such as specific 
transaction categories  with fees associated.

In [35]:
import re


In [37]:
import PyPDF2
import pandas as pd

In [39]:
!pip install PyPDF2 

In [40]:
from pdfminer.high_level import extract_pages,extract_text
for pages_layout in extract_pages("sample-invoice.pdf"):
    for element in pages_layout:
        print(element)

<LTTextBoxHorizontal(0) 58.560,673.518,274.814,680.478 'CPB Software (Germany) GmbH - Im Bruch 3 - 63897 Miltenberg/Main\n'>
<LTTextBoxHorizontal(1) 59.040,611.088,141.453,658.128 'Musterkunde AG\nMr. John Doe\nMusterstr. 23\n12345 Musterstadt\n'>
<LTTextBoxHorizontal(2) 364.870,611.492,446.685,619.532 'Name:  Stefanie Müller\n'>
<LTTextBoxHorizontal(3) 364.870,599.132,453.286,607.172 'Phone: +49 9371 9786-0\n'>
<LTTextBoxHorizontal(4) 59.040,536.538,194.556,546.498 'Invoice WMACCESS Internet\n'>
<LTTextBoxHorizontal(5) 58.800,513.062,143.027,521.102 'VAT No. DE199378386\n'>
<LTTextBoxHorizontal(6) 100.220,486.340,150.687,509.418 'Invoice No\n123100401\n'>
<LTTextBoxHorizontal(7) 213.290,486.820,275.859,509.898 'Customer No\n12345\n'>
<LTTextBoxHorizontal(8) 323.710,486.820,422.287,509.898 'Invoice Period\n01.02.2024 - 29.02.2024\n'>
<LTTextBoxHorizontal(9) 480.580,486.820,533.050,509.898 'Date\n1. März 2024\n'>
<LTTextBoxHorizontal(10) 138.860,456.542,213.801,464.582 'Service Descript

In [41]:
text = extract_text("sample-invoice.pdf")
print(text)

CPB Software (Germany) GmbH - Im Bruch 3 - 63897 Miltenberg/Main

Musterkunde AG
Mr. John Doe
Musterstr. 23
12345 Musterstadt

Name:  Stefanie Müller

Phone: +49 9371 9786-0

Invoice WMACCESS Internet

VAT No. DE199378386

Invoice No
123100401

Customer No
12345

Invoice Period
01.02.2024 - 29.02.2024

Date
1. März 2024

Service Description

Basic Fee wmView

Basis fee for additional user accounts

Basic Fee wmPos

Basic Fee wmGuide

Change of user accounts

Transaction Fee T1

Transaction Fee T2

Transaction Fee T3

Transaction Fee T4

Transaction Fee T5

Transaction Fee T6

Transaction Fee G1

Transaction Fee G2

Transaction Fee G3

Transaction Fee G4

Transaction Fee G5

Transaction Fee G6

Amount
-without VAT-

130,00 €

10,00 €

50,00 €

1.000,00 €

10,00 €

0,58 €

0,70 €

1,50 €

0,50 €

0,80 €

1,80 €

0,30 €

0,30 €

0,40 €

0,40 €

0,30 €

0,30 €

Total

VAT 19 %

Gross Amount incl. VAT

quantity

Total Amount

1

0

0

0

0

14

0

162

0

0

0

0

0

0

0

0

0

130,00 €

0

## Template Creation for Data Extraction

In [42]:
import json


In [43]:
header_template = {
    "Vendor Name": {"rule": "Find 'CPB Software (Germany) GmbH' as the vendor name"},
    "Customer Name": {"rule": "Find the customer name from the section starting with 'Musterkunde AG'"},
    "Invoice Number": {"rule": "Find 'Invoice No' and extract the numeric value following it"},
    "Invoice Date": {"rule": "Find 'Date' and extract the following date in DD/MM/YYYY or MM/DD/YYYY format"},
    "Customer Number": {"rule": "Find 'Customer No' and extract the following numeric value"},
    "Invoice Period": {
        "rule": "Find 'Period' and extract the start and end dates",
        "format": "Start Date - End Date"
    },
    "Total Amount (without VAT)": {"rule": "Find the 'Total Amount' without VAT and extract the value"},
    "VAT": {"rule": "Find 'VAT 19%' and extract the value"},
    "Gross Amount (with VAT)": {"rule": "Find 'Gross Amount incl. VAT' and extract the total"},
    "IBAN": {"rule": "Find 'IBAN' and extract the following alphanumeric string"},
    "BIC": {"rule": "Find 'BIC' and extract the following alphanumeric string"}
}

In [44]:
table_template = {
    "Line Items": {
        "rule": "Find the section with service descriptions and extract rows of line items",
        "columns": {
            "Description": {"rule": "Extract the text from the first column as item/service description"},
            "Quantity": {"rule": "Extract the number of items or services from the second column"},
            "Price": {"rule": "Extract the price per item/service from the third column"},
            "Total Price": {"rule": "Extract the total price for each line item"}
        },
        "multiple_rows": True 
    }
}

In [45]:
invoice_template = {
    "Header": header_template,
    "Table Data": table_template
}
json_template = json.dump(invoice_template, indent=4)
with open("invoice_template.json", 'w') as f:
    f.write(json_template)
print(json_template)

TypeError: dump() missing 1 required positional argument: 'fp'

In [46]:
invoice_template = {
    "Header": header_template,
    "Table Data": table_template
}
json_template = json.dumps(invoice_template, indent=4)
with open("invoice_template.json", 'w') as f:
    f.write(json_template)
print(json_template)

{
    "Header": {
        "Vendor Name": {
            "rule": "Find 'CPB Software (Germany) GmbH' as the vendor name"
        },
        "Customer Name": {
            "rule": "Find the customer name from the section starting with 'Musterkunde AG'"
        },
        "Invoice Number": {
            "rule": "Find 'Invoice No' and extract the numeric value following it"
        },
        "Invoice Date": {
            "rule": "Find 'Date' and extract the following date in DD/MM/YYYY or MM/DD/YYYY format"
        },
        "Customer Number": {
            "rule": "Find 'Customer No' and extract the following numeric value"
        },
        "Invoice Period": {
            "rule": "Find 'Period' and extract the start and end dates",
            "format": "Start Date - End Date"
        },
        "Total Amount (without VAT)": {
            "rule": "Find the 'Total Amount' without VAT and extract the value"
        },
        "VAT": {
            "rule": "Find 'VAT 19%' and extract the v

## Debugging and Improving Extraction Logic



In [48]:
word_to_num = {
    "zero": 0, "one": 1, "two": 2, "three": 3, "four": 4, "five": 5,
    "six": 6, "seven": 7, "eight": 8, "nine": 9, "ten": 10, "eleven": 11,
    "twelve": 12, "thirteen": 13, "fourteen": 14, "fifteen": 15, 
    "sixteen": 16, "seventeen": 17, "eighteen": 18, "nineteen": 19, 
    "twenty": 20, "thirty": 30, "forty": 40, "fifty": 50, "sixty": 60, 
    "seventy": 70, "eighty": 80, "ninety": 90, "hundred": 100, 
    "thousand": 1000, "million": 1000000
}

In [60]:
def words_to_number(words):
    words = words.lower().replace("-", " ").split()
    result = 0
    current = 0
    for word in words:
        if word in word_to_num:
            scale = word_to_num[word]
            if scale < 100:
                current += scale
            else:
                current *= scale
        elif word == "and":
            continue
        else:
            return None  
    return result + current if current else None


In [62]:
def extract_invoice_amount(value_content):
    value_content = value_content.lower()
    patterns = [
        r"total amount[:\s]*€?\$?(\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2}))",  
        r"amount due[:\s]*€?\$?(\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2}))",   
        r"amount[:\s]*€?\$?(\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2}))",       
        r"gross amount incl\. vat[:\s]*€?\$?(\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2}))", 
    ]
    

In [73]:
pattern = re.compile(r"[a-zA-Z]+,{1}\s{1}")
matches = pattern.findall(text)
print(matches)

['wmview, ']


In [92]:
   for pattern in patterns:
        match = re.search(pattern, value_content)
        if match:
            return match.group(1).replace(".", "").replace(",", ".")
    
    # Handling amounts in words, like "Amount: Five Hundred"
    words_pattern = r"amount[:\s]*(\w+.*\w+)"
    match = re.search(words_pattern, value_content)
    if match:
        amount_in_words = match.group(1)
        amount = words_to_number(amount_in_words)
        if amount is not None:
            return str(amount)
    
    
    return "N/A"

IndentationError: unindent does not match any outer indentation level (<string>, line 7)

In [96]:
def words_to_number(words):
    words = words.lower().replace("-", " ").split()
    result = 0
    current = 0
    
    for word in words:
        if word in word_to_num:
            scale = word_to_num[word]
            if scale < 100:
                current += scale
            else:
                current *= scale
                result += current  
                current = 0  
        elif word == "and":
            continue
        else:
            return None 
    
    return result + current if current else result 

In [98]:
test_cases = [
    "Total Amount: 381,12 €",
    "Amount Due: 500,00 €",
    "Amount: Five Hundred ",
    "Amount: 1.500,00 €",
    "Gross Amount incl. VAT: 1.000,25 €"
]

In [100]:
for test in test_cases:
    print(f"Input: {test} -> Extracted Amount: {extract_invoice_amount(test)}")

Input: Total Amount: 381,12 € -> Extracted Amount: None
Input: Amount Due: 500,00 € -> Extracted Amount: None
Input: Amount: Five Hundred  -> Extracted Amount: None
Input: Amount: 1.500,00 € -> Extracted Amount: None
Input: Gross Amount incl. VAT: 1.000,25 € -> Extracted Amount: None


##  Template Optimization and Efficiency

In [102]:
def create_general_rule(field_patterns):
    """
    Generate a regex-based rule that can match multiple variations of field names.
    Args:
    field_patterns (list of str): A list of possible patterns/field names.
    Returns:
    dict: A rule with a regex that matches any of the provided patterns.
    """
    joined_patterns = "|".join([re.escape(pattern) for pattern in field_patterns])
    return {"rule": f"Find any of {joined_patterns} and extract the following value"}


In [104]:
header_template = {
    "Vendor Name": create_general_rule(["Vendor", "Supplier", "Company"]),
    "Customer Name": create_general_rule(["Customer", "Client"]),
    "Invoice Number": create_general_rule(["Invoice No", "Invoice #", "Bill No"]),
    "Invoice Date": create_general_rule(["Date", "Invoice Date", "Billing Date"]),
    "Customer Number": create_general_rule(["Customer No", "Client ID", "Account No"]),
    "VAT Number": create_general_rule(["VAT No", "Tax ID"]),
    "Invoice Period": create_general_rule(["Period", "Invoice Period", "Billing Period"]),
    "Bank Details": {
        "IBAN": create_general_rule(["IBAN"]),
        "BIC": create_general_rule(["BIC", "Swift Code"])
    }
}


In [106]:
table_template = {
    "Table": {
        "Service Description": create_general_rule(["Description", "Service", "Item"]),
        "Quantity": create_general_rule(["Quantity", "Qty"]),
        "Unit Price": create_general_rule(["Unit Price", "Price per Item", "Cost"]),
        "Total Amount per Item": create_general_rule(["Total", "Amount", "Line Total"])
    }
}

In [108]:
invoice_template = {
    "Header": header_template,
    "Table": table_template
}


In [110]:
frequent_patterns = {
    "Vendor Name": re.compile(r"(vendor|supplier|company)", re.IGNORECASE),
    "Invoice Number": re.compile(r"(invoice no|invoice #|bill no)", re.IGNORECASE)
}


In [112]:
def extract_with_cache(field, text, pattern_cache):
    """
    Extract the data for a given field using cached regex patterns.

    Args:
    field (str): The field to extract (e.g., 'Invoice Number').
    text (str): The invoice text to search within.
    pattern_cache (dict): Cached regex patterns for common fields.

    Returns:
    str: Extracted value or 'N/A' if not found.
    """
    if field in pattern_cache:
        match = pattern_cache[field].search(text)
        if match:
            return match.group(1).strip()
    return "N/A"

In [114]:
template_json = json.dumps(invoice_template, indent=4)
with open("invoice_template_optimized.json", "w") as f:
    f.write(template_json)
print(template_json)

{
    "Header": {
        "Vendor Name": {
            "rule": "Find any of Vendor|Supplier|Company and extract the following value"
        },
        "Customer Name": {
            "rule": "Find any of Customer|Client and extract the following value"
        },
        "Invoice Number": {
            "rule": "Find any of Invoice\\ No|Invoice\\ \\#|Bill\\ No and extract the following value"
        },
        "Invoice Date": {
            "rule": "Find any of Date|Invoice\\ Date|Billing\\ Date and extract the following value"
        },
        "Customer Number": {
            "rule": "Find any of Customer\\ No|Client\\ ID|Account\\ No and extract the following value"
        },
        "VAT Number": {
            "rule": "Find any of VAT\\ No|Tax\\ ID and extract the following value"
        },
        "Invoice Period": {
            "rule": "Find any of Period|Invoice\\ Period|Billing\\ Period and extract the following value"
        },
        "Bank Details": {
            "IBAN": 

## Dealing with Complex Scenarios 

In [116]:
def extract_text_from_pdf(file_path):
    page_1 = "Invoice No: 12345\nDate: 01.03.2024\nVendor: CPB Software\n"
    page_2 = "Service Description | Quantity | Unit Price | Total\nBasic Service | 1 | 100.00 € | 100.00 €\nExtended Service | Continued on next line\nDescription | 2 | 150.00 € | 300.00 €"
    return [page_1, page_2]


In [118]:

def process_multi_page_invoice(text_pages):
    extracted_data = {}
    header_page = text_pages[0]
    
    try:
        extracted_data['Invoice Number'] = re.search(r'Invoice No:\s*(\d+)', header_page).group(1)
    except AttributeError:
        extracted_data['Invoice Number'] = "N/A"
        
    try:
        extracted_data['Invoice Date'] = re.search(r'Date:\s*([0-9]{2}\.[0-9]{2}\.[0-9]{4})', header_page).group(1)
    except AttributeError:
        extracted_data['Invoice Date'] = "N/A"

    try:
        extracted_data['Vendor Name'] = re.search(r'Vendor:\s*(.+)', header_page).group(1)
    except AttributeError:
        extracted_data['Vendor Name'] = "N/A"

    return extracted_data


In [122]:
 table_data = []
    For page in text_pages:
        try:
            rows = re.findall(r'([a-zA-Z\s]+)\|\s*(\d+)\s*\|\s*([0-9,\.]+ €)\s*\|\s*([0-9,\.]+ €)', page)
            for row in rows:
                table_data.append({
                    "Description": row[0].strip(),
                    "Quantity": row[1].strip(),
                    "Unit Price": row[2].strip(),
                    "Total": row[3].strip()
                })
        except Exception as e:
            print(f"Error while extracting table data on this page: {str(e)}")

IndentationError: unexpected indent (3132780931.py, line 2)

In [124]:
def extract_table_data(text_pages):
    table_data = []
    
    for page in text_pages:
        try:
            rows = re.findall(r'([a-zA-Z\s]+)\|\s*(\d+)\s*\|\s*([0-9,\.]+ €)\s*\|\s*([0-9,\.]+ €)', page)
            
            for row in rows:
                table_data.append({
                    "Description": row[0].strip(),
                    "Quantity": row[1].strip(),
                    "Unit Price": row[2].strip(),
                    "Total": row[3].strip()
                })
                
            if not rows:
                print("No rows found on this page.")
                
        except Exception as e:
            print(f"Error while extracting table data on this page: {str(e)}")
    
    return table_data

In [134]:
 def process_table_data(table_data, extracted_data):
    processed_table_data = []
    i = 0

    while i < len(table_data):
        if "Continued" in table_data[i]["Description"]:
            if i + 1 < len(table_data):
    
                merged_row = {
                    "Description": table_data[i]["Description"].replace("Continued", "").strip() + " " + table_data[i + 1]["Description"].strip(),
                    "Quantity": table_data[i + 1]["Quantity"],
                    "Unit Price": table_data[i + 1]["Unit Price"],
                    "Total": table_data[i + 1]["Total"]
                }
                processed_table_data.append(merged_row)
               
                i += 2
            else:
                print("Warning: Last row marked 'Continued' but no row to merge with.")
                i += 1
        else:
            processed_table_data.append(table_data[i])
            i += 1

    extracted_data['Table'] = processed_table_data
    return extracted_data



In [136]:
def extract_with_error_handling(text, field_name, regex_pattern):
    try:
        return re.search(regex_pattern, text).group(1)
    except AttributeError:
        print(f"Error: {field_name} not found, returning fallback value 'N/A'")
        return "N/A"

pdf_text = extract_text_from_pdf("sample-invoice.pdf")
extracted_invoice_data = process_multi_page_invoice(pdf_text)
print(json.dumps(extracted_invoice_data, indent=4))
invoice_text = "Sample Invoice\nDate: Missing"
invoice_date = extract_with_error_handling(invoice_text, "Invoice Date", r"Date:\s*([0-9]{2}\.[0-9]{2}\.[0-9]{4})")
print(f"Extracted Invoice Date: {invoice_date}")

{
    "Invoice Number": "12345",
    "Invoice Date": "01.03.2024",
    "Vendor Name": "CPB Software"
}
Error: Invoice Date not found, returning fallback value 'N/A'
Extracted Invoice Date: N/A
